## Chapter 6: Multiset functions

### Joint evaluation

Chaining pool methods only goes so far. For one, it only gives you one evaluation of a time. What if you want to perform multiple evaluations jointly? For example, what if you wanted to know both the largest matching set and the largest straight resulting from a pool?

To do this, you can use the `@multiset_function` decorator. Similar to how `@map_function` applies a function to all possible outcomes that could come out of a die (or dice), you can think of `@multiset_function` applying a function to all multisets that could come out of a pool (or pools). However, `@multiset_function` only supports a limited set of methods and operators, and the result of the function decorated by `@multiset_function` must be a multiset evaluation; or a tuple of them, which will be evaluated jointly. In exchange, it is able to use an algorithm that is typically much more efficient than *actually* enumerating all possible multisets. You can see the chapter on evaluators for details.

In [1]:
import piplite
await piplite.install("icepool")

from icepool import multiset_function, d6

@multiset_function
def yahtzee(a):
    return a.largest_count(), a.largest_straight()

print(yahtzee(d6.pool(6)))

Die with denominator 46656

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          1 |          6 |      720 |   1.543210% |
|          2 |          1 |      360 |   0.771605% |
|          2 |          2 |     7560 |  16.203704% |
|          2 |          3 |    10440 |  22.376543% |
|          2 |          4 |     6840 |  14.660494% |
|          2 |          5 |     3600 |   7.716049% |
|          3 |          1 |     1640 |   3.515089% |
|          3 |          2 |     7300 |  15.646433% |
|          3 |          3 |     4320 |   9.259259% |
|          3 |          4 |     1440 |   3.086420% |
|          4 |          1 |      660 |   1.414609% |
|          4 |          2 |     1230 |   2.636317% |
|          4 |          3 |      360 |   0.771605% |
|          5 |          1 |      120 |   0.257202% |
|          5 |          2 |       60 |   0.128601% |
|          6 |          1 |        6 |   0.012860% |




Note how this captures the dependence between the largest count and largest straight---if you have a straight of 6 on 6 dice, then they surely all rolled different numbers, so the largest count must be 1.

### Multiple parameters

The function can take in multiple parameters, as long as the number of parameters is fixed. For example, this computes how many unique outcomes each of two pools has that the other doesn't:

In [2]:
@multiset_function
def unique_mutual_difference(a, b):
    return (a.unique() - b.unique()).count(), (b.unique() - a.unique()).count()

print(unique_mutual_difference(d6.pool(4), d6.pool(4)))

Die with denominator 1679616

| Outcome[0] | Outcome[1] | Quantity | Probability |
|-----------:|-----------:|---------:|------------:|
|          0 |          0 |    37506 |   2.233010% |
|          0 |          1 |    82500 |   4.911837% |
|          0 |          2 |    32400 |   1.929012% |
|          0 |          3 |     1440 |   0.085734% |
|          1 |          0 |    82500 |   4.911837% |
|          1 |          1 |   325950 |  19.406221% |
|          1 |          2 |   247080 |  14.710505% |
|          1 |          3 |    42480 |   2.529150% |
|          1 |          4 |      720 |   0.042867% |
|          2 |          0 |    32400 |   1.929012% |
|          2 |          1 |   247080 |  14.710505% |
|          2 |          2 |   302760 |  18.025549% |
|          2 |          3 |    82080 |   4.886831% |
|          2 |          4 |     5040 |   0.300069% |
|          3 |          0 |     1440 |   0.085734% |
|          3 |          1 |    42480 |   2.529150% |
|          3 |  

### Bound generators

Other than the arguments to the function, any other pools mentioned in the evaluator are considered to be independent, and are bound when `multiset_function` is invoked. For example:

In [3]:
target = [1, 2, 3]

@multiset_function
def early_binding_example(a):
    return (a & target).count()

print(early_binding_example(d6.pool(3)))

target = [1]
print(early_binding_example(d6.pool(3)))

Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |       27 |  12.500000% |
|       1 |      111 |  51.388889% |
|       2 |       72 |  33.333333% |
|       3 |        6 |   2.777778% |


Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |       27 |  12.500000% |
|       1 |      111 |  51.388889% |
|       2 |       72 |  33.333333% |
|       3 |        6 |   2.777778% |




The mention of `target` inside `early_binding_example` is already bound, so changing it afterwards doesn't change the result. Another example:

In [4]:
from icepool import d12

target = d12.pool(1)

# Remember, since we are dealing with multisets here, the `>=` operator means `issuperset`.
@multiset_function
def two_vs_target(a, b):
    return a >= target, b >= target

print(two_vs_target(d6.pool(6), (d6 + 6).pool(6)))

Die with denominator 313456656384

| Outcome[0] | Outcome[1] |     Quantity | Probability |
|:-----------|:-----------|-------------:|------------:|
| False      | False      | 139641226596 |  44.548815% |
| False      | True       |  69575101596 |  22.196084% |
| True       | False      |  69575101596 |  22.196084% |
| True       | True       |  34665226596 |  11.059018% |




Here the roll of the `target` d12 is not an argument to the evaluator, so is rolled independently for each of its two comparisons in the function. Therefore is no correlation between the two elements of the result. So this might as well have been two independent calculations. If we wanted to use the same roll for both comparisons, we could make it an argument:

In [5]:
@multiset_function
def two_vs_target(a, b, t):
    return a >= t, b >= t

print(two_vs_target(d6.pool(6), (d6 + 6).pool(6), d12.pool(1)))

Die with denominator 26121388032

| Outcome[0] | Outcome[1] |   Quantity | Probability |
|:-----------|:-----------|-----------:|------------:|
| False      | False      | 8748000000 |  33.489798% |
| False      | True       | 8686694016 |  33.255101% |
| True       | False      | 8686694016 |  33.255101% |




Now the same roll of the d12 is used for both comparisons; as we can see, at most one pool can contain the target number, since the first pool only rolls from 1 to 6 and the second from 7 to 12.